In [1]:
import numpy as np
import os
import os.path

# Set up matplotlib and use a nicer set of plot parameters
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import sep
import pylab

from astropy.io import fits
from astropy.wcs import WCS

%matplotlib inline

In [2]:
def get_data (path):
    img_data = fits.open(path+'img.fits')[0].data[400:800,400:800]
    sig_data = fits.open(path+'sig.fits')[0].data[400:800,400:800]
    msk_data = fits.open(path+'bad.fits')[0].data[400:800,400:800]
    
    return img_data,sig_data,msk_data

In [3]:
import os
import shutil

directories = os.listdir('hsc_images')

for directory in directories:
    #ignore any files that may have been added on accident
    if not str.isdigit(directory[0]):
        continue
    
    num = directory
    path = 'hsc_images/'+num+'/redadd_'+num+'_HSC-I_full_'
    
    img_data, sig_data, msk_data = get_data(path)
    
    # Change the byteorder (translate the bytes)
    img_data = img_data.byteswap().newbyteorder()
    sig_data = sig_data.byteswap().newbyteorder()
    msk_data = msk_data.byteswap().newbyteorder()
    
    #step 2
    bkg_model = sep.Background(img_data, mask=msk_data, bw=32, bh=32, fw=3, fh=3)
    img_sub = img_data - bkg_model.back()
    
    #step 3
    obj_table, seg_data = sep.extract(img_sub, 5.0, err=sig_data, mask=msk_data, 
                                      minarea=5, deblend_nthresh=32, deblend_cont=0.001,
                                      clean=True, segmentation_map=True)
    
    #step 4
    #gets elliptical approximations of the ends of the galaxies, create visualizations
    fig, ax = plt.subplots()
    img_avg, img_std = np.mean(img_sub), np.std(img_sub)

    im = ax.imshow(img_sub, interpolation='nearest',
                   vmin=(img_avg - img_std), vmax=(img_avg + img_std), 
                   origin='lower')

    for i in range(len(obj_table)):
        e = Ellipse(xy=(obj_table['x'][i], 
                        obj_table['y'][i]),
                    width=(6 * obj_table['a'][i]),
                    height=(6 * obj_table['b'][i]),
                    angle=(obj_table['theta'][i] * 180. / np.pi))
        e.set_facecolor('none')
        e.set_edgecolor('red')
        ax.add_artist(e)
    
    #move figure to respective folders for organization
    #pylab.savefig('eliptical_image_'+num,bbox_inches='tight')
    #src = os.path.abspath('eliptical_image_'+num+'.png')
    #dst = os.path.abspath(path+'img.fits')
    #shutil.move(src,dst)
    
    

OSError: Header missing END card.